# WHISPER

`Whisper` es un modelo de reconocimiento de voz de propósito general. Está entrenado en un gran conjunto de datos de audio diverso (680.000 horas de audios transcritos) y también es un modelo multitarea que puede realizar reconocimiento de voz multilingüe, así como traducción de voz e identificación de idioma.

## Acerca del modelo

Whisper es un modelo de Transformer sequence-to-sequence entrenado en varias tareas de procesamiento de voz, incluido el reconocimiento de voz multilingüe, la traducción de voz, la identificación del idioma hablado y la detección de actividad de voz. Todas estas tareas se representan conjuntamente como una secuencia de tokens que el decodificador debe predecir, lo que permite que un solo modelo reemplace muchas etapas diferentes de una canalización de procesamiento de voz tradicional. El formato de entrenamiento multitarea utiliza un conjunto de tokens especiales que sirven como especificadores de tareas u objetivos de clasificación.

<img src="https://github.com/openai/whisper/blob/main/approach.png?raw=true"><br>

En este [paper](https://cdn.openai.com/papers/whisper.pdf) se puede leer un poco mas de su desarrollo. 

## Configuración

Primero, hay que asegurarse de estar usando un entorno de ejecución GPU para ejecutar este notebook, de modo que la inferencia sea mucho más rápida. Si el siguiente comando falla, use el menú `entorno de ejecución` del menu y seleccione `Cambiar tipo de entorno ejecución`.

In [ ]:
!nvidia-smi

Sat Nov 19 12:57:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Se uso Python 3.9.9 y PyTorch 1.10.1 para entrenar y probar el modelo, pero se espera que el código base sea compatible con Python 3.7 o versiones posteriores y recientes de PyTorch. El código base también depende de algunos paquetes de Python, sobre todo [HuggingFace Transformers](https://huggingface.co/docs/transformers/index) para su rápida implementación de tokenizer y [ffmpeg-python](https://github.com/kkroening/ffmpeg-python) para leer archivos de audio. El siguiente comando extraerá e instalará la última confirmación de este repositorio, junto con sus dependencias de Python

In [ ]:
!pip install git+https://github.com/openai/whisper.git 
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-fxlfepuj
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-fxlfepuj
     |████████████████████████████████| 5.5 MB 33.2 MB/s 
     |████████████████████████████████| 7.6 MB 62.4 MB/s 
     |████████████████████████████████| 182 kB 73.4 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175239 sha256=69d9a44bc6220b6297048b4cce1b75716d287109912201bf457cee28295e09e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-sosl9zdz/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 44.6 MB/s 
     |████████████████████████████████| 2.2 MB 56.8 MB/s 


Es posible que necesite instalar otros paquetes como rust o ffmpeg. Si lo solicita, ejecutando la siguiente linea de codigo se instalan

In [ ]:
# !pip install setuptools-rust
# !pip install ffmpeg-python

## Modelos e idiomas disponibles
Hay cinco tamaños de modelos, cuatro con versiones solo en inglés, que ofrecen ventajas y desventajas de velocidad y precisión. A continuación se muestran los nombres de los modelos disponibles y sus requisitos aproximados de memoria y velocidad relativa.

| Size	| Parameters	| English-only model	| Multilingual model	| Required VRAM	| Relative speed |
| :-----: | :-----: | :-----: | :-----: | :-----: | :-----: |
|tiny	|39 M	|tiny.en	|tiny	|~1 GB	|~32x |
|base	|74 M	|base.en	|base	|~1 GB	|~16x |
|small	|244 M	|small.en	|small	|~2 GB	|~6x |
|medium	|769 M	|medium.en	|medium	|~5 GB	|~2x |
|large	|1550 M	|N/A	|large	|~10 GB	|1x |

Para aplicaciones solo en inglés, los modelos `.en`  tienden a funcionar mejor, especialmente para los modelos `tiny.en` y `base.en`. Se observa que la diferencia se vuelve menos significativa para los modelos `small.en` y `medium.en`

## Uso en Python

In [ ]:
#@title Ejecutar esta celda para cargar la funcion para grabar audio.

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Presiona para comenzar a grabar");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Grabando... presione para detener";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Guardando la grabación... por favor espere!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

### Importamos las librerias necesarias

In [ ]:
import numpy as np
import whisper
from scipy.io.wavfile import write
from IPython.display import clear_output


### Grabar audio

In [ ]:
audio, sr = get_audio()
name = 'record.wav'
write(name, sr, audio)

### Aplicamos Whisper

Como primer tarea vamos a tomar el audio que grabamos y transcribirlo.
El metodo `transcribe()` se enacrga de transcribir un archivo de audio usando Whisper

In [ ]:
model = whisper.load_model("medium")
result = model.transcribe(name)

clear_output()
print("-- SU TRANSCRIPCIÓN --\n")
print('{}\n'.format(result["text"]))
print("Idioma detectado: {}".format(result['language']))



-- SU TRANSCRIPCIÓN --

 Solo transcribir audio a idioma original con whisper.

Idioma detectado: es


Si queremos guardar la transcripcion:

In [ ]:
archivo_txt = name.split()[0] + '.txt'
with open(archivo_txt, 'w', encoding="utf-8") as t:
    t.write(result["text"])

Internamente, el método `transcribe()` lee todo el archivo y procesa el audio con una ventana deslizante de 30 segundos, realizando predicciones autorregresivas de secuencia a secuencia en cada ventana.

## Uso a un nivel mas inferior
Probemos ahora el uso de whisper.`detect_language()` y `whisper.decode()` que proporcionan acceso de nivel inferior al modelo.

In [ ]:
# cargar audio y rellenarlo/recortarlo para que se ajuste a 30 segundos
audio = whisper.load_audio(name)
audio = whisper.pad_or_trim(audio)

# creamos el espectrograma log-Mel y lo enviamos al dispositivo
mel = whisper.log_mel_spectrogram(audio).to(model.device)


Metodo `detect_language()` Detecta el idioma hablado en el audio y lo devuelve como una lista de strings, junto con los identificadores de los tokens de idioma más probables y la distribución de probabilidad sobre todos los tokens de idioma

In [ ]:
# detectar el idioma hablado
_, probs = model.detect_language(mel)
print(f"Idioma detectado: {max(probs, key=probs.get)}")

Idioma detectado: es


In [ ]:
# decodificar el audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Solo transcribir audio a idioma original con whisper.


## Traducir de español a ingles

Para traducir usamos el metodo `transcribe()` aclarando que queremos que desarrolle la tarea de traduccion con:  `task="translate"`

In [ ]:
# Cargamos el modelo
model = whisper.load_model("medium")

# Cargamos el audio
audio = whisper.load_audio(name)

# Le pasamos las tareas
transcription = model.transcribe(audio)["text"] # por defecto task='transcribe'
translation = model.transcribe(audio, task="translate")["text"]
    

In [ ]:
print('transcription: {}'.format(transcription))
print('translation: {}'.format(translation))


transcription:  Solo transcribir audio a idioma original con whisper.
translation:  Just transcribe audio to original language with whisper
